In [177]:
#This block of code is used to get the data of National Highways and their embeded links

from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
html = urlopen('https://en.wikipedia.org/wiki/List_of_National_Highways_in_India_by_highway_number')
bsObj = BeautifulSoup(html, 'lxml')
A = []
B = []
C = []
D = []
E = []
F = []
G = []
H = []
for data in bsObj.body.find('table', {'class' : 'wikitable sortable'}).findAll('tr'):
    cells = data.findAll('td')
    if data.find('td') is None:
        pass
    else:
        A.append('National Highway ' + cells[0].find(text = True) + ' (India)')
        B.append(cells[1].find(text = True))
        C.append(cells[2].find(text = True))
        D.append(cells[3].find(text = True))
        E.append(cells[4].find(text = True))
        F.append('https://en.wikipedia.org' + cells[0].find('a').attrs['href'])
        if cells[1].find('a') is None:
            G.append('None')
        else:
            G.append(cells[1].find('a').attrs['href'])
        #for url in data.findAll('a', href = re.compile('^(/wiki/National_Highway)')):
            #A.append(url.attrs['title'])
            #F.append(url.attrs['href'])
        #for url in data.findAll('a', href = re.compile('^(/wiki/)(?!National)')):
            #G.append(url.attrs['href'])
            #H.append(url.text)

#import pandas to convert list to data frame
import pandas as pd
df=pd.DataFrame(A,columns=['National Highways'])
df['OSMRI'] = B #OSMRI : Open Street Map Relation Identifier
df['States'] = C
df['length'] = D
df['routes'] = E
df['National Highway URLS'] = F
df['OSMRI_URL'] = G
#print (df)
df.to_csv('National_Highways.csv',index=True,header=True)

In [58]:
"""
This block of code is used to extract urls from http://badroadsinindia.com/ go to the every url and download 
the images of road maps of each state from http://www.mapsofindia.com/statename
"""

from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import requests
html = urlopen('http://badroadsinindia.com/')
soup = BeautifulSoup(html, 'lxml')

try:
    def download_image(url,name):
        res = requests.get(url)
        #res.raise_for_status()
        image = open(name,'wb')
        for chunk in res.iter_content(100000):
            image.write(chunk)
        image.close()

    A = []
    name = []
    states = []
    first_urls = []
    complete_urls = []
    for data in soup.findAll('a', href = re.compile('^(http://www.mapsofindia.com)')):
        A.append(data.attrs['href'])
        states.append(data.text.lower().replace(' ','-'))
    for link in A:
        url_string = link.split('/')
        needed_url = '/'.join(url_string[ : -1])
        first_urls.append(needed_url)
    for i in range(len(A)):
        complete_urls.append(first_urls[i] + '/' + states[i] + '-road-map.jpg')
    for i in range(len(complete_urls)):
        name.append(str(states[i]).upper().replace('-',' ') + ' roads.jpg')
        download_image(complete_urls[i], name[i])
except requests.exceptions.HTTPError as err:
    print (err)

In [ ]:
#Code for Dandegous_Roads_In_India

from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
import pandas as pd
import re

#Using requests we can download html file of a web page

req = requests.get('http://www.indiatimes.com/culture/travel/15-dangerous-roads-in-india-that-are-a-driver’s-worst-nightmare-245762.html')
req.raise_for_status()
road_woes = open('India_times.html', 'wb')
for chunk in req.iter_content(100000):
    road_woes.write(chunk)
road_woes.close()

try:
    html = open('/home/damodhar/MyPythonScripts/Assignment2/India_times.html')
    soup = BeautifulSoup(html, 'lxml')
    A = []
    B = []
    #C = []
    name_of_road = []
    
    for data in (soup.find('div', {'class' : 'div-fixed'}).findAll('h3')):
        A.append(data.text)
        
    for data in (soup.find('div', {'class' : 'div-fixed'}).findAll('p')):
        if data.find('img', src = re.compile('^(http)')) is None:
            if (data.find('iframe', src = re.compile('^(http)'))) is None:
                pass
            else:
                B.append(data.find('iframe', src = re.compile('^(http)'))['src'])
        else:
            B.append(data.find('img', src = re.compile('^(http)'))['src'])
    
    #list index out of range(Urls are the mix of images and videos and the videos doesn't contain image credit)
    #for data in (soup.find('div', {'class' : 'div-fixed'}).find('p').findAll('span', {'class' : 'source_link'})):
        #C.append(data.text.replace('Image Credit: ', ''))
        
    for i in range(len(A)):
        string = A[i].split(' ')
        name_of_road.append(' '.join(string[1 : ]))
        download_image(B[i], name_of_road[i] + '.jpg')
    
    def download_image(url, name):
        req = requests.get(url)
        req.raise_for_status()
        image = open(name, 'wb')
        for chunk in req.iter_content(100000):
            image.write(chunk)
        image.close()
except requests.exceptions.HTTPError as err:
    print (err)

df=pd.DataFrame(name_of_road,columns=['Road Name'])
df["Road's image Url"] = B
#df['Image Credit'] = C
df.to_csv('Dangerous_Roads_In_India.csv',index=True,header=True)

In [43]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

req = requests.get('http://www.badroadsinindia.com/')
req.raise_for_status()
road_woes = open('road_woes.html', 'wb')
for chunk in req.iter_content(100000):
    road_woes.write(chunk)
road_woes.close()

In [26]:
import requests
from bs4 import BeautifulSoup

req = requests.get('https://en.wikipedia.org/wiki/List_of_National_Highways_in_India_by_highway_number')
req.raise_for_status()
nh_roads = open('nh_roads.html', 'wb')
for chunk in req.iter_content(100000):
    nh_roads.write(chunk)
nh_roads.close()